Importing Libraries

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

Training Image Preprocessing

In [ ]:
training_set = tf.keras.utils.image_dataset_from_directory(
    'train',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

Validition Image Preprocessing (to track the accuracy and performance of the model)

In [ ]:
validation_set=tf.keras.utils.image_dataset_from_directory(
    'valid',
    labels="inferred",#take label from the file name
    label_mode="categorical",#as it is multi-class problem means more than option to choose otherwise use int or binary
    class_names=None,
    color_mode="rgb",
    batch_size=32,#speed of training process can be 64 or 128
    image_size=(128, 128),
    shuffle=True,#reduce the biasness of model randomly choose any to match
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
pad_to_aspect_ratio=False,
    verbose=True,
)

Note:To avoid overshooting
1.Choose small leaning rate deafault,0.001 we are taking 0.0001
2.There may be a chance of underfitting, so increase number of neuron 
3.And more conovlution layer to extract more feature from images there may be possibility model unable to capture relevant feature or model is confusing due to lack of feature sowith more feature

Building Model

In [ ]:
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout
from tensorflow.keras.models import Sequential
model=Sequential()
#Building Convotional layer
model.add(Conv2D(filters=32,kernel_size=3,padding='same',activation='relu',input_shape=[128,128,3]))
model.add(Conv2D(filters=32,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

model.add(Conv2D(filters=64,kernel_size=3,padding='same',activation='relu',input_shape=[128,128,3]))
model.add(Conv2D(filters=64,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

model.add(Conv2D(filters=128,kernel_size=3,padding='same',activation='relu',input_shape=[128,128,3]))
model.add(Conv2D(filters=128,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

model.add(Conv2D(filters=256,kernel_size=3,padding='same',activation='relu',input_shape=[128,128,3]))
model.add(Conv2D(filters=256,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

model.add(Conv2D(filters=512,kernel_size=3,padding='same',activation='relu',input_shape=[128,128,3]))
model.add(Conv2D(filters=512,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

model.add(Dropout(0.25))#To avoid overfitting
model.add(Flatten())
model.add(Dense(units=1500,activation='relu'))
model.add(Dropout(0.4))

#Output Layer
model.add(Dense(units=38,activation='softmax'))


Compilling Model

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model Training

In [ ]:
training_history=model.fit(x=training_set,validation_data=validation_set,epochs=10)



Model Evaluation

In [ ]:
#Note: Model Evaluation on training 
train_loss,train_acc=model.evaluate(training_set)
print(train_loss,train_acc)

val_loss,val_acc=model.evaluate(validation_set)
print(val_loss,val_acc)

Saving Model

In [ ]:
model.save("trained_model.keras")
print(training_history.history)

#Recording History in JSON
import json
with open("training_hist.json","w") as f:
    json.dump(training_history.history,f)

Accuracy Visualization

In [ ]:
epochs=[i for i in range(1,11)]
plt.plot(epochs,training_history.history['accuracy'],color='red',label='Training Accuracy')
plt.plot(epochs,training_history.history['val_accuracy'],color='blue',label='Validation Accuracy')
plt.xlabel("Number of Epochs")
plt.ylabel("Accuracy Result")
plt.title("Visualization of Accuracy Result")
plt.legend()
plt.show()

Some metrics for model evaluation

In [ ]:
class_name=validation_set.class_names
class_name

test_set=tf.keras.utils.image_dataset_from_directory(
    'train',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)
y_pred=model.predict(test_set)
y_pred=y_pred.shape
predicted_categories=tf.argmax(y_pred,axis=1)
predicted_categories

true_categories=tf.concat([y for x,y in test_set],axis=0)
true_categories

Y_true=tf.argmax(true_categories,axis=1)
Y_true

from sklearn.metrics import classification_report,confusion_matrix
classification_report(Y_true,predicted_categories,target_names=class_name)

cm=confusion_matrix(Y_true,predicted_categories)
cm

Confusion Matrix Visualization

In [ ]:
plt.figure(figsize=(40,40))
sns.heatmap(cm,annot=True,annot_kws=('size':10))
plt.xlabel("Predicted Class",fontsize=20)
plt.ylabel("Actual Class",fontsize=20)
plt.title("Plant disease Prediction Confusion Matrix",fontsize=25)
plt.show()